## k Nearest Neighber

가장 가까운 k개의 이웃 데이터가 어떤 레이블인지 보고, 다수결(혹은 평균)로 새 샘플의 레이블을 결정

In [16]:
# 필요한 라이브러리 임포트
# - 데이터 로드/분할/스케일링/KNN 및 파이프라인/GridSearch에서 사용
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# 1) 데이터 로드: iris 데이터셋 사용
# X: 특성 행렬, y: 레이블(클래스)
iris = load_iris()
X, y = iris.data, iris.target  # 이후 코드에서 쓰기 편하게 분리

# 2) 학습/평가용 분할
# stratify=y를 사용하면 클래스 비율을 유지한 채로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=13,
    stratify=y
)

# 3) 파이프라인 정의
# KNN은 거리 기반 알고리즘이라 스케일링(표준화)이 중요합니다.
# 따라서 StandardScaler와 KNeighborsClassifier를 파이프라인으로 묶습니다.
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

# 이제 이후 셀에서 GridSearchCV에 이 pipe를 전달하면 스케일링과 모델 학습
# 이 일관되게 적용되어 하이퍼파라미터 탐색이 가능합니다.

In [17]:
# 간단히 KNN을 직접 학습해보는 예제 (파이프라인 없이)
# 참고: 실제 하이퍼파라미터 탐색에서는 아래의 pipe + GridSearchCV 방식을 권장합니다.
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# 간단 평가 (학습 데이터에 대한 성능 확인)
train_score = knn.score(X_train, y_train)
print(f"간단 학습 점수 (train): {train_score:.4f}")


간단 학습 점수 (train): 0.9750


In [18]:
# 하이퍼파라미터 탐색: 파이프라인의 knn 부분 파라미터를 튜닝
# 'knn__n_neighbors'처럼 접두사로 파이프라인 단계 이름을 붙여서 지정합니다.
param_grid = {
    'knn__n_neighbors': [1,3,5,7,9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

# GridSearchCV 객체 생성: pipe를 기반으로 그리드 탐색
# cv=5로 5-겹 교차검증 사용, n_jobs=-1로 가능한 모든 CPU 코어 사용
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("CV best score:", grid.best_score_)


Best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance'}
CV best score: 0.9666666666666666


In [19]:
# 테스트 세트에 대한 최적 모델 평가
# grid.best_estimator_는 학습된 파이프라인(스케일러+knn)을 반환합니다.
best = grid.best_estimator_
y_pred = best.predict(X_test)

print("=== Test set evaluation ===")
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

# 참고: confusion_matrix로 클래스별 오분류 현황을 확인하고,
# classification_report로 정밀도/재현율/F1-score를 확인하세요.

=== Test set evaluation ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Confusion matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
